In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("EPL Analysis") \
    .getOrCreate()
df = spark.read.option("header", "true").csv("EPL.csv")

print("Schema:")
df.printSchema()
print("\nSample data:")
df.show(5)

df = df.withColumn("home_goals", col("home_goals").cast("float")) \
       .withColumn("away_goals", col("away_goals").cast("float"))

goals_by_season = df.withColumn("total_goals", col("home_goals") + col("away_goals")) \
                   .groupBy("season") \
                   .agg(sum("total_goals").alias("total_goals")) \
                   .orderBy(desc("total_goals"))

print("Seasons with highest number of goals:")
goals_by_season.show()

highest_goals_season = goals_by_season.first()
print(f"Season with highest goals: {highest_goals_season['season']} with {highest_goals_season['total_goals']} goals")

lowest_goals_season = goals_by_season.orderBy("total_goals").first()
print(f"Season with lowest goals: {lowest_goals_season['season']} with {lowest_goals_season['total_goals']} goals")
home_teams = df.select(col("home_team").alias("team"), col("home_goals").alias("goals"), "season")
away_teams = df.select(col("away_team").alias("team"), col("away_goals").alias("goals"), "season")

all_teams_goals = home_teams.union(away_teams)


team_stats = all_teams_goals.groupBy("team") \
    .agg(
        sum("goals").alias("total_goals"),
        countDistinct("season").alias("seasons_played")
    ) \
    .withColumn("avg_goals_per_season", col("total_goals") / col("seasons_played")) \
    .filter(col("seasons_played") > 0) \
    .orderBy(desc("avg_goals_per_season"))

print("Teams with highest average goals per season:")
team_stats.show()

highest_avg_team = team_stats.first()
print(f"Team with highest average goals: {highest_avg_team['team']} with {highest_avg_team['avg_goals_per_season']:.2f} goals per season")


man_utd_matches = df.filter(
    (col("home_team") == "Manchester United") | (col("away_team") == "Manchester United")
)


man_utd_results = man_utd_matches.withColumn(
    "man_utd_result",
    when(col("home_team") == "Manchester United",
         when(col("result") == "H", "Win")
         .when(col("result") == "A", "Lose")
         .otherwise("Draw")
    ).otherwise(
        when(col("result") == "A", "Win")
        .when(col("result") == "H", "Lose")
        .otherwise("Draw")
    )
)


result_counts = man_utd_results.groupBy("man_utd_result") \
    .agg(count("*").alias("count"))

total_matches = man_utd_results.count()

probabilities = result_counts.withColumn(
    "probability",
    col("count") / total_matches
)

print("Manchester United result probabilities:")
probabilities.show()


win_prob = probabilities.filter(col("man_utd_result") == "Win").select("probability").first()
lose_prob = probabilities.filter(col("man_utd_result") == "Lose").select("probability").first()
draw_prob = probabilities.filter(col("man_utd_result") == "Draw").select("probability").first()

print(f"\nManchester United Probabilities:")
print(f"P(Win) = {win_prob['probability']:.3f}" if win_prob else "P(Win) = 0.000")
print(f"P(Lose) = {lose_prob['probability']:.3f}" if lose_prob else "P(Lose) = 0.000")
print(f"P(Draw) = {draw_prob['probability']:.3f}" if draw_prob else "P(Draw) = 0.000")


rdd = df.rdd
print(f"\nRDD created with {rdd.count()} rows")

spark.stop()

Schema:
root
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_goals: string (nullable = true)
 |-- away_goals: string (nullable = true)
 |-- result: string (nullable = true)
 |-- season: string (nullable = true)


Sample data:
+----------------+----------------+----------+----------+------+---------+
|       home_team|       away_team|home_goals|away_goals|result|   season|
+----------------+----------------+----------+----------+------+---------+
|Sheffield United|       Liverpool|       1.0|       1.0|     D|2006-2007|
|         Arsenal|     Aston Villa|       1.0|       1.0|     D|2006-2007|
|         Everton|         Watford|       2.0|       1.0|     H|2006-2007|
|Newcastle United|  Wigan Athletic|       2.0|       1.0|     H|2006-2007|
|      Portsmouth|Blackburn Rovers|       3.0|       0.0|     H|2006-2007|
+----------------+----------------+----------+----------+------+---------+
only showing top 5 rows

Seasons with highest number of